<a href="https://colab.research.google.com/github/ku839060/CSMBD21-CloudComputing-/blob/main/CSMBD21(Cloud_computing_coursework).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [336]:
import pandas as pd
import csv

# Reading the datasets to datastructures, 
Pandas was used for this case for usability this also allowed for the checking of the results to ensure that they where correct.

In [337]:
AComp_passenger_data_no_errorDF = pd.read_csv("/content/drive/MyDrive/University of reading/Sem 2/Big data and cloud computing/Cloud computing CourseWork/Data/AComp_Passenger_data_no_error.csv", header=None)
Top30_airports_LatLongDF = pd.read_csv("/content/drive/MyDrive/University of reading/Sem 2/Big data and cloud computing/Cloud computing CourseWork/Data/Top30_airports_LatLong.csv", header=None)

In [366]:
AComp_passenger_data_no_errorDF
#checking the dataset using pandas for visual aid.

,0,1,2,3,4,5
0,UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049
1,UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802
2,EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250
3,ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250
4,CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322
...,...,...,...,...,...,...
495,BWI0520BG6,BER7172M,KUL,LAS,1420565167,1848
496,LLZ3798PE3,EWH6301Y,CAN,DFW,1420564967,1683
497,KKP5277HZ7,KJR6646J,IAH,BKK,1420565203,1928
498,JJM4724RF7,XXQ4064B,JFK,FRA,1420563917,802


# Reading and parsing the data

In [340]:
#opening the dataset
AComp_passenger_data_no_error = open('/content/drive/MyDrive/University of reading/Sem 2/Big data and cloud computing/Cloud computing CourseWork/Data/AComp_Passenger_data_no_error.csv')
#Top30_airports_LatLong = open('/content/drive/MyDrive/University of reading/Sem 2/Big data and cloud computing/Cloud computing CourseWork/Data/Top30_airports_LatLong.csv')

#reading the dataset
AComp_passenger_data_no_error_reader = csv.reader(AComp_passenger_data_no_error)
#Top30_airports_LatLong_reader = csv.reader(Top30_airports_LatLong)

#Parseing the data. for use within the mapper.
parsedata = []
for row in AComp_passenger_data_no_error_reader:
  parsedata.append(row)



# Mapper Phase Class

In [348]:
class Mapper(object):
  def __init__(self): super(Mapper, self).__init__()
  
  #the mapper function that assigns a key and a value to a list containing tuples.
  def mapper(self, datastructure, colum):
    TupleList = []
    for line in datastructure:
      TupleList.append(tuple( (line[colum], 1)))
    return TupleList


# Multithreading class functions

As the threading inbuilt does not allow for easy return of functions this function was used to facilitate this aspect.

In [352]:
from threading import Thread

#this class allows for the returning of functions when using threading
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,args=(), kwargs={}, Verbose=None):
      Thread.__init__(self, group, target, name, args, kwargs)
      self._return = None

  #sets up the thread
    def run(self):
        print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    #initalises the thread.
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

Multithreading mapper function to ensure that all the threads are completed before moving onto the reducing stage.

In [353]:
def multithreadingmapper(datastructure,column,chunks):

  #breaking the data into chunks
  n = int((len(parsedata))/chunks)
  chunkeddata=[parsedata[i:i + n] for i in range(0, len(parsedata), n)]

  #creating threads for each of the chunks for the mapper to utilise
  #in this case four where used.
  listofmappedchunks = []
  for chunk in chunkeddata:
      chunkmappedthread = ThreadWithReturnValue(target = MapperObject.mapper,args = (chunk,column))
      chunkmappedthread.start()
      listofmappedchunks.append(chunkmappedthread.join())

  #combining the results of the threads into one single which can then be utilised by the reducer.
  Combinedmappedresult = []
  for mappedchunk in listofmappedchunks:
       Combinedmappedresult = Combinedmappedresult + mappedchunk

  return Combinedmappedresult


# Reduce Phase class

In [363]:
class Reducer(object):  
  def __init__(self):
          super(Reducer, self).__init__()

  # this function provides a reduced dictionary for use in the lower functions
  def reduct(self,TupleList):
    reductdict = {}
    for key, value in TupleList:
      reductdict[key] = reductdict.get(key,0) + value
    return reductdict

  #when standard reducing this sorts them by value as well
  def reducer(self,TupleList):

    reducerDict = self.reduct(TupleList)
    sorted_tuples = sorted(reducerDict.items(), key=lambda item: item[1],reverse = True)
    sorted_dict = {k: v for k, v in sorted_tuples}

    return list(sorted_dict.items())

  #this function returns only the highest key value, Used for highest number of flights
  def HighestReducer(self,TupleList):

    reducerDict = {}
    reducerDict = self.reduct(TupleList)
    HighestPassenger = max(reducerDict, key=reducerDict.get)

    return HighestPassenger

# Creating a mapper and reducer object

In [364]:
MapperObject = Mapper()
ReducerObject = Reducer()

# Task 1

In [365]:
#passing the colum related to flight from this is where multiple nodes of the mapper are used in threads
Combinedmappedresult = multithreadingmapper(parsedata,2,4)

#reducing the mapped results down. this function also sorts into decending order.
ReducerOutput = ReducerObject.reducer(Combinedmappedresult)

ReducerOutput

<class 'method'>
<class 'method'>
<class 'method'>
<class 'method'>


[('DEN', 46),
 ('CAN', 37),
 ('IAH', 37),
 ('ATL', 36),
 ('ORD', 33),
 ('KUL', 33),
 ('CGK', 27),
 ('JFK', 25),
 ('LHR', 25),
 ('CDG', 21),
 ('CLT', 21),
 ('PVG', 20),
 ('LAS', 17),
 ('BKK', 17),
 ('AMS', 15),
 ('FCO', 15),
 ('MUC', 14),
 ('MAD', 13),
 ('PEK', 13),
 ('HND', 13),
 ('DFW', 11),
 ('MIA', 11)]

# task 2

In [356]:
#passing the colum related to Passengers from this is where multiple nodes of the mapper are used in threads
Combinedmappedresult = multithreadingmapper(parsedata,0,4)
#returning only the passenger that coccurs most frequently.
ReducerHighestOutput = ReducerObject.HighestReducer(Combinedmappedresult)
ReducerHighestOutput

<class 'method'>
<class 'method'>
<class 'method'>
<class 'method'>


'UES9151GS5'